In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Features relevant to predicting price of a listing

In [192]:
columns = ["Country", "City", "State", "Neighbourhood Cleansed", "Neighbourhood Group Cleansed",
"Host Since", "Host Response Time", "Host Response Rate", "Calculated host listings count",
"Property Type", "Room Type", "Accommodates", "Bedrooms", "Beds", "Bed Type",  "Square Feet", "Cancellation Policy",
"Minimum Nights", "Maximum Nights", "Has Availability", "Availability 30", "Availability 60", "Availability 90", "Availability 365",
"Number of Reviews", "Reviews per Month", "First Review", "Last Review", "Review Scores Rating", "Review Scores Accuracy", "Review Scores Cleanliness","Review Scores Checkin", "Review Scores Communication", "Review Scores Location","Review Scores Value",
"Features",
"Amenities",
"Price"]

# "Square Feet": 97.56% of entries have NULL values => cannot be used
#"Calendar Updated"
#"Weekly Price", "Monthly Price", "Security Deposit", "Cleaning Fee", "Guests Included", "Extra People",

features_new_host = ["Country", "City", "Neighbourhood Cleansed",
                   "Property Type", "Room Type", "Accommodates","Bedrooms", "Beds", "Bed Type", "Cancellation Policy",
                   "Minimum Nights",
                   "Price"]

features_existing_host = ["Country", "City", "Neighbourhood Cleansed",
                          "Property Type", "Room Type", "Accommodates", "Bedrooms", "Beds", "Bed Type", "Cancellation Policy",
                          "Minimum Nights", "Availability 30", "Availability 60", "Availability 90", "Availability 365",
                          "Number of Reviews", "Reviews per Month", "Review Scores Rating", "Review Scores Accuracy", "Review Scores Cleanliness","Review Scores Checkin", "Review Scores Communication", "Review Scores Location","Review Scores Value",
                          "Host Since", "Host Response Time", "Host Response Rate", "Calculated host listings count",
                          "Price"]

############################################
########## Feature pre-processing ##########
#### Numerical (Input text fields)
# "Minimum Nights", "Availability 30", "Availability 60", "Availability 90", "Availability 365"
# "Number of Reviews", 
# Review Scores Rating(20-100)
# "Host Since" (num_days = current_date - host_since_date), Host Response Rate(0-100), "Calculated host listings count"
# Price (Convert from local currency to USD)

#### Numerical (Dropdown numerical values)
# Accommodates(1-16), "Bedrooms"(0-10), "Beds"(0-16)
# "Review Scores Accuracy(2-10)", "Review Scores Cleanliness(2-10)","Review Scores Checkin(2-10)", "Review Scores Communication(2-10)", "Review Scores Location(2-10)","Review Scores Value(2-10)"

#### Label encoding (0, 1, 2 .... n_categories-1) (Category dropdowns -> on user select must be assigned numerical value)
# "Country", "City", "Neighbourhood Cleansed"
# "Property Type", "Room Type", "Bed Type", "Cancellation Policy"
# "Host Response Time"
############################################


# TODO
# Host since -> convert to days ----------------------------------- Done
# Features -> Extract features
# Amenities -> Extract features
# Price -> Convert to local currency
# Verify missing features (Listing type etc...) ------------------- Done
# Decide what plots to be shown in exploratory analysis section ---
# Flask vs Flast + React ------------------------------------------ Done (Flask)

In [193]:
df = pd.read_csv("../../data/airbnb-data-science/airbnb-listings.csv", usecols=columns, sep=';')
df = df[columns]
df.head(2)

/Users/nesara/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (66) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Country,City,State,Neighbourhood Cleansed,Neighbourhood Group Cleansed,Host Since,Host Response Time,Host Response Rate,Calculated host listings count,Property Type,...,Review Scores Rating,Review Scores Accuracy,Review Scores Cleanliness,Review Scores Checkin,Review Scores Communication,Review Scores Location,Review Scores Value,Features,Amenities,Price
0,Netherlands,Amsterdam,Noord-Holland,Watergraafsmeer,NaN,2017-03-28,NaN,NaN,1.0,Apartment,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Host Has Profile Pic,"TV,Kitchen,Heating,Washer,Smoke detector,Lapto...",80.0
1,Netherlands,Amsterdam,Noord-Holland,Watergraafsmeer,NaN,2014-10-21,within a day,100.0,1.0,House,...,94.0,9.0,9.0,10.0,9.0,10.0,9.0,"Host Has Profile Pic,Host Identity Verified,Is...","TV,Wireless Internet,Kitchen,Heating,Family/ki...",195.0


In [196]:
#dfo = df.copy()
len(dfo)

485419

In [142]:
dfo["First Review"].unique()

array([nan, '2015-05-09', '2016-12-04', ..., '2010-05-06', '2010-05-12',
       '2009-09-03'], dtype=object)

In [202]:
dfo[pd.isnull(dfo["Accommodates"])]

,Country,City,State,Neighbourhood Cleansed,Neighbourhood Group Cleansed,Host Since,Host Response Time,Host Response Rate,Calculated host listings count,Property Type,...,Review Scores Rating,Review Scores Accuracy,Review Scores Cleanliness,Review Scores Checkin,Review Scores Communication,Review Scores Location,Review Scores Value,Features,Amenities,Price
6766,United States,Denver,CO,City Park West,NaN,2012-12-09,within a day,86.0,7.0,House,...,97.0,10.0,10.0,10.0,10.0,10.0,9.0,"Host Has Profile Pic,Host Identity Verified,Is...","TV,Cable TV,Internet,Wireless Internet,Air Con...",120.0
6772,United States,Denver,CO,Lincoln Park,NaN,2014-07-01,within an hour,100.0,1.0,Condominium,...,93.0,10.0,10.0,9.0,9.0,9.0,9.0,"Host Has Profile Pic,Host Identity Verified,Is...","TV,Internet,Wireless Internet,Air Conditioning...",200.0
23598,United States,Denver,CO,Sunnyside,NaN,2015-06-16,NaN,NaN,1.0,House,...,100.0,10.0,10.0,10.0,10.0,10.0,10.0,"Host Has Profile Pic,Host Identity Verified,Is...","TV,Cable TV,Internet,Wireless Internet,Kitchen...",80.0
23600,United States,Denver,CO,Sunnyside,NaN,2015-02-19,within a few hours,100.0,1.0,House,...,91.0,10.0,9.0,10.0,10.0,10.0,9.0,"Host Has Profile Pic,Host Identity Verified,Is...","Cable TV,Wireless Internet,Air Conditioning,Ki...",179.0
23602,United States,Denver,CO,Sunnyside,NaN,2012-09-15,within a few hours,90.0,2.0,House,...,95.0,10.0,10.0,10.0,10.0,9.0,10.0,"Host Has Profile Pic,Host Identity Verified,Is...","TV,Cable TV,Internet,Wireless Internet,Air Con...",230.0
23606,United States,Denver,CO,Highland,NaN,2015-06-03,NaN,NaN,1.0,House,...,80.0,8.0,8.0,10.0,10.0,8.0,8.0,"Host Has Profile Pic,Host Identity Verified,Is...","TV,Cable TV,Internet,Wireless Internet,Air Con...",100.0
23607,United States,Denver,CO,Highland,NaN,2013-12-03,within a day,92.0,2.0,House,...,98.0,10.0,10.0,10.0,10.0,10.0,10.0,"Host Has Profile Pic,Host Identity Verified,Is...","Wireless Internet,Air Conditioning,Kitchen,Ind...",169.0
23608,United States,Denver,CO,Highland,NaN,2016-02-04,within a few hours,100.0,1.0,House,...,100.0,10.0,10.0,10.0,10.0,9.0,8.0,"Host Has Profile Pic,Host Identity Verified,Is...","TV,Cable TV,Internet,Wireless Internet,Air Con...",215.0
23720,United States,Denver,CO,Five Points,NaN,2013-12-27,within a day,90.0,1.0,Apartment,...,92.0,10.0,10.0,10.0,10.0,9.0,9.0,"Host Has Profile Pic,Host Identity Verified,Is...","Air Conditioning,Kitchen,Free Parking on Premi...",150.0
40666,United States,Denver,CO,Five Points,NaN,2014-05-19,within a day,77.0,1.0,House,...,99.0,10.0,10.0,10.0,10.0,10.0,10.0,"Host Is Superhost,Host Has Profile Pic,Host Id...","TV,Internet,Wireless Internet,Air Conditioning...",96.0


In [203]:
dfo.groupby("Accommodates").count()

,Country,City,State,Neighbourhood Cleansed,Neighbourhood Group Cleansed,Host Since,Host Response Time,Host Response Rate,Calculated host listings count,Property Type,...,Review Scores Rating,Review Scores Accuracy,Review Scores Cleanliness,Review Scores Checkin,Review Scores Communication,Review Scores Location,Review Scores Value,Features,Amenities,Price
Accommodates,,,,,,,,,,,,,,,,,,,,,
1.0,42547,42479,38303,42548,13238,42472,28437,28437,42457,42546,...,26210,26071,26098,26044,26100,26051,26036,42498,41661,42419
2.0,206294,206097,185115,206295,45423,206052,152426,152426,205863,206289,...,154585,154198,154302,154057,154305,154075,154033,206152,204105,205413
3.0,46296,46246,41411,46296,10544,46238,36008,36008,46249,46296,...,35955,35888,35912,35859,35899,35860,35861,46269,45965,45813
4.0,97522,97453,87160,97522,17737,97456,77458,77458,97384,97520,...,75162,75013,75049,74942,75036,74956,74940,97481,96970,95851
5.0,24311,24292,21508,24311,4625,24296,19869,19869,24296,24311,...,18706,18670,18676,18656,18676,18661,18652,24300,24199,23482
6.0,37145,37125,33414,37145,6152,37122,31396,31396,37080,37145,...,27393,27342,27362,27316,27344,27320,27306,37126,36948,35792
7.0,6898,6898,6151,6898,1084,6894,5899,5899,6890,6898,...,5091,5079,5084,5081,5084,5079,5079,6898,6870,6593
8.0,12600,12587,11569,12600,1739,12587,11008,11008,12568,12600,...,8684,8663,8672,8658,8673,8659,8657,12591,12531,11721
9.0,2125,2122,1914,2125,318,2125,1900,1900,2122,2125,...,1513,1510,1510,1507,1510,1508,1508,2125,2115,1958


## Data cleaning

### Existing hosts

In [258]:
df = dfo.copy()

########################################################
############ Location Features #############
###### Countries ######## 
# Remove countries with very few entries
exclude_low_count_countries_list = ["0", "Cuba", "Mexico", "Uruguay", "Vanuatu", "Vatican City"]
df = df[~df["Country"].isin(exclude_low_count_countries_list)]
# Remove null (3 entries)
df = df[~pd.isnull(df["Country"])]

###### City ######## 
# Remove null (451 entries)
df = df[~pd.isnull(df["City"])]


###### Neighbourhood Cleansed ######
# Remove null (1 entry)
df = df[~pd.isnull(df["Neighbourhood Cleansed"])]
#######################################################


####################################################################
############ Basic (Beds, Area, Cancellation) Features #############

###### Property Type #####
# Remove null (11 entries)
df = df[~pd.isnull(df["Property Type"])]
# Remove types with less than 10 entries
df = df.groupby("Property Type").filter(lambda x: len(x)>=10)

###### Room Type #####
# Remove null (1 entries)
df = df[~pd.isnull(df["Room Type"])]

###### Accommodates #####
# Remove null (63 entries)
df = df[~pd.isnull(df["Accommodates"])]
# Accommodate maximum 16 people
df = df[df["Accommodates"] <= 16]

###### Bedrooms #########
# Remove null (597 entries)
df = df[~pd.isnull(df["Bedrooms"])]
# Remove more than 10 bedrooms
df = df[df["Bedrooms"] <= 10]

###### Beds #########
# Remove null (903 entries)
df = df[~pd.isnull(df["Beds"])]
# Remove more than 16 beds (2 entries)
df = df[df["Beds"] <= 16]

###### Bed Type #########
# Remove null (1 entry)
df = df[~pd.isnull(df["Bed Type"])]

###### Cancellation Policy #########
# Remove cancellation policy with low entries (3)
exclude_cancellation_policy_list = ["long_term", "no_refunds"]
df = df[~df["Cancellation Policy"].isin(exclude_cancellation_policy_list)]
# Remove null (2 entries)
df = df[~pd.isnull(df["Cancellation Policy"])]
####################################################################


##########################################################
######### Availability #########

###### Minimum Nights #########
# Remove "Minimum Nights" above 30 except for multiples of 30 (corresponding to months)
df = df[(df["Minimum Nights"]<=31) | (df["Minimum Nights"].isin([60, 90, 120, 180]))]
# Remove null (2 entries)
df = df[~pd.isnull(df["Minimum Nights"])]

###### Availability 30 #########
# Remove null (2 entries)
df = df[~pd.isnull(df["Availability 30"])]

###### Availability 60 #########
# Remove null (2 entries)
df = df[~pd.isnull(df["Availability 60"])]

###### Availability 90 #########
# Remove null (2 entries)
df = df[~pd.isnull(df["Availability 90"])]

###### Availability 365 #########
# Remove null (2 entries)
df = df[~pd.isnull(df["Availability 365"])]
##########################################################


##########################################################
######### Reviews #########

###### Number of Reviews #########
# Remove entries with 0 reviews (Use Model for new host in this case)
df = df[df["Number of Reviews"] > 0]
# Remove null (2 entries)
df = df[~pd.isnull(df["Number of Reviews"])]

###### Reviews per Month #########
#df.at[(df["Number of Reviews"]==0) & (df[pd.isnull("Reviews per Month")]), "Reviews per Month"] = 0
# Remove null (235 entries)
df = df[~pd.isnull(df["Reviews per Month"])]

##### Reviews Scores #############
df = df[~pd.isnull(df["Review Scores Rating"])]
df = df[~pd.isnull(df["Review Scores Accuracy"])]
df = df[~pd.isnull(df["Review Scores Cleanliness"])]
df = df[~pd.isnull(df["Review Scores Checkin"])]
df = df[~pd.isnull(df["Review Scores Communication"])]
df = df[~pd.isnull(df["Review Scores Location"])]
df = df[~pd.isnull(df["Review Scores Value"])]
##########################################################



##########################################################
######### Host Features #########

###### Host Since #########
# Remove null (504 entries)
df = df[~pd.isnull(df["Host Since"])]

# Convert date to number of days till today
today_date = pd.to_datetime(pd.to_datetime("today").date())
df["Host Since"] = pd.to_datetime(df["Host Since"])
df["Host Since Days"] = (today_date - df["Host Since"])
df['Host Since Days'] = df[['Host Since Days']].apply(pd.to_numeric)
df['Host Since Days'] = df['Host Since Days'] / (24*60*60*1e9) #np.timedelta64(1, 'D')
df["Host Since Days"] = df["Host Since Days"].astype(int)

"""
df["Host Since Days"] = 0
for i in range(len(df)):
    idx = df.index[i]
    df.at[idx, "Host Since Days"] = (today_date - df.iloc[idx]["Host Since"]).days
"""

###### Host Response Time #####
# Assign "a few days or more" to entries with NULL values for "Host Response Time"
df.at[pd.isnull(df["Host Response Time"]), "Host Response Time"] = "a few days or more"

###### Host Response Rate #####
# Assign mean(93.4) to entries with NULL values for "Host Response Rate"
df.at[pd.isnull(df["Host Response Rate"]), "Host Response Rate"] = dfo["Host Response Rate"].mean()

###### Calculated host listings count #####
# Assign 1 to entries with NULL values for "Calculated host listings count"
df.at[pd.isnull(df["Calculated host listings count"]), "Calculated host listings count"] = 1
##########################################################


In [259]:
len(df)

356297

In [250]:
df["Host Since"] = pd.to_datetime(df["Host Since"])
df["Host Since Days"] = (today_date - df["Host Since"])
df['Host Since Days'] = df[['Host Since Days']].apply(pd.to_numeric)
df['Host Since Days'] = df['Host Since Days'] / (24*60*60*1e9) #np.timedelta64(1, 'D')
df["Host Since Days"] = df["Host Since Days"].astype(int)
df["Host Since Days"]

1         2175
2         2209
4         2999
7         1705
8         1906
9         1754
10        1660
11        2334
13        2962
14        2517
15        2918
16        2321
17        2900
18        1630
19        2236
20        2362
21        2628
22        2362
23        2035
24        2259
25        3246
26        1552
27        2513
29        1833
30        2380
31        2726
32        1418
33        3073
34        1721
36        2337
          ... 
485370    1989
485371    1851
485372    1970
485375    2694
485376    3100
485377    1604
485378    1826
485379    2365
485381    2189
485383    1762
485384    1971
485385    1717
485386    1717
485391    1438
485395    1657
485396    2332
485397    2014
485398    3044
485399    2315
485402    2058
485403    1831
485405    1769
485406    2554
485408    1581
485410    2643
485412    1983
485413    1652
485414    1617
485415    2455
485416    2013
Name: Host Since Days, Length: 356297, dtype: int64

In [236]:
len(df)

356297

In [204]:
dfo["Host Since"]

0         2017-03-28
1         2014-10-21
2         2014-09-17
3         2016-06-15
4         2012-07-19
5         2016-01-26
6         2013-10-31
7         2016-02-03
8         2015-07-17
9         2015-12-16
10        2016-03-19
11        2014-05-15
12        2013-10-04
13        2012-08-25
14        2013-11-13
15        2012-10-08
16        2014-05-28
17        2012-10-26
18        2016-04-18
19        2014-08-21
20        2014-04-17
21        2013-07-25
22        2014-04-17
23        2015-03-10
24        2014-07-29
25        2011-11-15
26        2016-07-05
27        2013-11-17
28        2016-09-28
29        2015-09-28
             ...    
485389    2015-02-12
485390    2017-03-06
485391    2016-10-27
485392    2015-02-12
485393    2016-02-11
485394    2016-03-28
485395    2016-03-22
485396    2014-05-17
485397    2015-03-31
485398    2012-06-04
485399    2014-06-03
485400    2014-05-17
485401    2017-03-11
485402    2015-02-15
485403    2015-09-30
485404    2014-12-20
485405    201

In [209]:
pd.to_datetime("today").date()

datetime.date(2020, 10, 4)

In [210]:
df["Host Since"] = pd.to_datetime(df["Host Since"])

In [212]:
pd.to_datetime(pd.to_datetime("today").date())

Timestamp('2020-10-04 00:00:00')

In [230]:
dft = pd.DataFrame()
today_date = pd.to_datetime(pd.to_datetime("today").date())
dft["Host Since"] = df["Host Since"].copy()
#dft["test"] = (pd.to_datetime(pd.to_datetime("today").date()) - df["Host Since"])
dft.index[0]

0

In [225]:
dft

,Host Since
0,2017-03-28
1,2014-10-21
2,2014-09-17
3,2016-06-15
4,2012-07-19
5,2016-01-26
6,2013-10-31
7,2016-02-03
8,2015-07-17
9,2015-12-16


In [71]:
len(df) / len(dfo)

0.9963351249127043

In [117]:
len(dfo[(dfo["Minimum Nights"]<=31) | (dfo["Minimum Nights"].isin([60, 90, 120, 180]))]) / len(dfo)

0.9982736563669737

In [107]:
len(dfo[(dfo["Maximum Nights"]<=31) | (dfo["Maximum Nights"].isin([60, 90, 120, 180]))])

142957

In [106]:
dfo[(dfo["Maximum Nights"]<=31) | (dfo["Maximum Nights"].isin([60, 90, 120, 180]))].groupby("Maximum Nights").count()

,Country,City,State,Neighbourhood Cleansed,Neighbourhood Group Cleansed,Host Since,Host Response Time,Host Response Rate,Calculated host listings count,Bedrooms,...,Review Scores Rating,Review Scores Accuracy,Review Scores Cleanliness,Review Scores Checkin,Review Scores Communication,Review Scores Location,Review Scores Value,Features,Amenities,Price
Maximum Nights,,,,,,,,,,,,,,,,,,,,,
1.0,551,550,510,551,101,550,332,332,550,550,...,290,289,289,290,290,289,290,550,541,533
2.0,1653,1651,1518,1653,337,1649,952,952,1650,1649,...,930,924,928,925,928,926,924,1650,1641,1623
3.0,4009,4003,3738,4009,866,4004,2535,2535,4000,4000,...,2408,2398,2398,2389,2398,2391,2387,4005,3995,3942
4.0,3732,3729,3404,3732,848,3727,2406,2406,3726,3727,...,2405,2397,2400,2397,2399,2396,2395,3729,3711,3645
5.0,5488,5483,4985,5488,1312,5480,3817,3817,5484,5482,...,3772,3761,3766,3753,3766,3752,3753,5483,5467,5411
6.0,2947,2945,2645,2947,763,2945,1909,1909,2946,2944,...,2001,1997,1998,1992,1996,1991,1990,2946,2936,2895
7.0,13488,13478,11945,13488,2808,13471,9600,9600,13473,13473,...,9370,9347,9349,9333,9347,9335,9330,13474,13446,13209
8.0,2532,2532,2235,2532,635,2528,1676,1676,2531,2530,...,1809,1805,1807,1801,1808,1803,1803,2532,2524,2494
9.0,1473,1472,1290,1473,396,1468,856,856,1473,1473,...,935,932,933,932,931,930,931,1470,1466,1446


In [ ]:
from pandas.plotting import scatter_matrix

attributes = ["Review Scores Rating", "Price"]
scatter_matrix(df[attributes], figsize=(12, 8))